A neural network consist of cnn layer (Yoon Kim,2014) and 4 fully connected layers.

Source: https://github.com/jojonki/cnn-for-sentence-classification





In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/sharif/DeepLearning/ipython(guide)')

In [12]:
import numpy as np
import codecs
import os
import random
import pandas
from keras import backend as K
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Lambda, Permute, Dropout
from keras.layers import Conv2D, MaxPooling1D
from keras.optimizers import SGD
import ast
import re
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import gensim
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [31]:
limit_number = 750
data = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv',index_col=0)
data = data.dropna().reset_index(drop=True)
X = data["body"].values.tolist()
y = pandas.read_csv('../Data/limited_to_'+str(limit_number)+'.csv')
labels = []
tag=[]
for item in y['tag']:
  labels += [i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' ']
  tag.append([i for i in re.sub('\"|\[|\]|\'| |=','',item.lower()).split(",") if i!='' and i!=' '])
labels = list(set(labels))
mlb = MultiLabelBinarizer()
Y=mlb.fit_transform(tag)

In [32]:
len(labels)

78

In [33]:
sentence_maxlen = max(map(len, (d for d in X)))
print('sentence maxlen', sentence_maxlen)

sentence maxlen 4404


In [47]:
freq_dist = pandas.read_csv('../Data/FreqDist_sorted.csv',index_col=False)
vocab=[]
for item in freq_dist["word"]:
  try:
    word=re.sub(r"[\u200c-\u200f]","",item.replace(" ",""))
    vocab.append(word)
  except:
    pass
  
print(vocab[10])

زبان


In [48]:
vocab = sorted(vocab)
vocab_size = len(vocab)

In [49]:
print('vocab size', len(vocab))
w2i = {w:i for i,w in enumerate(vocab)}
# i2w = {i:w for i,w in enumerate(vocab)}
print(w2i["زبان"])

vocab size 225345
129280


In [50]:
def vectorize(data, sentence_maxlen, w2i):
    vec_data = []
    
    for d in data:
       
        vec = [w2i[w] for w in d if w in w2i]
        pad_len = max(0, sentence_maxlen - len(vec))
        vec += [0] * pad_len
        vec_data.append(vec)
        # print(d)
        
    vec_data = np.array(vec_data)
    
    return vec_data

vecX = vectorize(X, sentence_maxlen, w2i)
vecY=Y

In [51]:
X_train, X_test, y_train, y_test = train_test_split(vecX, vecY, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
print('train: ', X_train.shape , '\ntest: ', X_test.shape , '\nval: ', X_val.shape ,"\ny_tain:",y_train.shape )
# print(vecX[0])

train:  (12896, 512) 
test:  (4299, 512) 
val:  (4299, 512) 
y_tain: (12896, 78)


In [52]:
embd_dim = 300


# ***If the word2vec model is not generated before, we should run the next block.***

In [53]:
# embed_model = gensim.models.Word2Vec(X, size=embd_dim, window=5, min_count=5)
# embed_model.save('word2vec_model')

# ***Otherwise, we can run the next block.***

In [54]:
embed_model=gensim.models.Word2Vec.load('word2vec_model')

In [55]:
word2vec_embd_w = np.zeros((vocab_size, embd_dim))
for word, i in w2i.items():
  if word in embed_model.wv.vocab:
        embedding_vector =embed_model[word]
  
        # words not found in embedding index will be all-zeros.
        word2vec_embd_w[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [56]:
from keras.layers import LSTM
def Net(vocab_size, embd_size, sentence_maxlen, glove_embd_w):
    sentence = Input((sentence_maxlen,), name='SentenceInput')
    
    # embedding
    embd_layer = Embedding(input_dim=vocab_size, 
                           output_dim=embd_size, 
                           weights=[word2vec_embd_w], 
                           trainable=False,
                           name='shared_embd')
    embd_sentence = embd_layer(sentence)
    embd_sentence = Permute((2,1))(embd_sentence)
    embd_sentence = Lambda(lambda x: K.expand_dims(x, -1))(embd_sentence)
    
    # cnn
    cnn = Conv2D(1, 
                 kernel_size=(5, sentence_maxlen),
                 activation='relu')(embd_sentence)
    cnn =  Lambda(lambda x: K.sum(x, axis=3))(cnn)
    cnn = MaxPooling1D(3)(cnn)
    cnn = Lambda(lambda x: K.sum(x, axis=2))(cnn)
    
    hidden1=Dense(400,activation="relu")(cnn)
    hidden2=Dense(300,activation="relu")(hidden1)
    hidden3=Dense(200,activation="relu")(hidden2)
    out = Dense(len(labels), activation='sigmoid')(hidden3)
    
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = Model(inputs=sentence, outputs=out, name='sentence_claccification')
    model.compile(optimizer=sgd, loss='binary_crossentropy',metrics=["accuracy"]) 

    return model

# sentence_maxlen = 512
model = Net(vocab_size, embd_dim, sentence_maxlen,word2vec_embd_w)
print(model.summary())

Model: "sentence_claccification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SentenceInput (InputLayer)   [(None, 512)]             0         
_________________________________________________________________
shared_embd (Embedding)      (None, 512, 300)          67603500  
_________________________________________________________________
permute_3 (Permute)          (None, 300, 512)          0         
_________________________________________________________________
lambda_9 (Lambda)            (None, 300, 512, 1)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 296, 1, 1)         2561      
_________________________________________________________________
lambda_10 (Lambda)           (None, 296, 1)            0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 98, 1) 

In [57]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50) # Model stop training after 50 epoch where validation loss didnt decrease
mc = ModelCheckpoint('best_cnn_4fc.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True) #You save model weight at the epoch where validation loss is minimal
model.fit(X_train, y_train, batch_size=32,epochs=250,verbose=1,validation_data=(X_val, y_val),callbacks=[es,mc])#you can run for 1000 epoch btw model will stop after 50 epoch without better validation loss

Epoch 1/250
403/403 [==============================] - 99s 243ms/step - loss: 0.6673 - accuracy: 0.0246 - val_loss: 0.5914 - val_accuracy: 0.0321

Epoch 00001: val_loss improved from inf to 0.59140, saving model to best_cnn_4fc.h5
Epoch 2/250
403/403 [==============================] - 95s 236ms/step - loss: 0.5697 - accuracy: 0.0166 - val_loss: 0.5108 - val_accuracy: 0.0121

Epoch 00002: val_loss improved from 0.59140 to 0.51079, saving model to best_cnn_4fc.h5
Epoch 3/250
403/403 [==============================] - 95s 235ms/step - loss: 0.4939 - accuracy: 0.0133 - val_loss: 0.4479 - val_accuracy: 0.0121

Epoch 00003: val_loss improved from 0.51079 to 0.44787, saving model to best_cnn_4fc.h5
Epoch 4/250
403/403 [==============================] - 95s 236ms/step - loss: 0.4342 - accuracy: 0.0305 - val_loss: 0.3982 - val_accuracy: 0.0121

Epoch 00004: val_loss improved from 0.44787 to 0.39820, saving model to best_cnn_4fc.h5
Epoch 5/250
403/403 [==============================] - 94s 234ms

# ***If the model is generated before:***

In [ ]:
# model = load_model('CNN_1_just_accuracy_-Dense.h5')
model.save('CNN_1_just_accuracy_-Dense.h5')

In [ ]:
pred=model.predict(X_test)
# For evaluation: If the probability > 0.5 you can say that it belong to the class.

In [ ]:
print(pred[0])#example

In [ ]:
y_pred=[]
measure = .3#*(np.mean(pred[0]) + .5*np.sqrt(np.var(pred[0])))
for l in pred:
  temp=[]
  for value in l:
    if value >= measure:
      temp.append(1)
    else:
      temp.append(0)
  y_pred.append(temp)

In [ ]:
from sklearn.metrics import classification_report,accuracy_score

print("accuracy=",accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))